In [ ]:
import pandas as pd
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers.gromacs import get_gromacs_energies
from openff.interchange.drivers.openmm import _get_openmm_energies, get_openmm_energies

In [ ]:
SMILES = "c1n(CCO)c(C(F)(F)(F))cc1CNCCl"

In [ ]:
molecule = Molecule.from_smiles(SMILES)
molecule.generate_conformers(n_conformers=20, rms_cutoff=0.1 * unit.angstrom),
topology = molecule.to_topology()

In [ ]:
parsley = ForceField("openff-1.1.0.offxml")

In [ ]:
openff_sys = Interchange.from_smirnoff(force_field=parsley, topology=topology)
openmm_sys = parsley.create_openmm_system(topology)

In [ ]:
openff_sys.positions = molecule.conformers[0]
openff_sys.box = [4, 4, 4]

In [ ]:
df = pd.DataFrame()

kj_mol = unit.kilojoule / unit.mol

for idx, conformer in enumerate(molecule.conformers):
    openff_sys.positions = conformer

    toolkit = sum(
        _get_openmm_energies(
            omm_sys=openmm_sys,
            box_vectors=openff_sys.box,
            positions=openff_sys.positions,
            platform="Reference",
        ).energies.values()
    ).m_as(kj_mol)

    omm = sum(get_openmm_energies(openff_sys).energies.values()).m_as(kj_mol)

    gmx = sum(get_gromacs_energies(openff_sys).energies.values()).m_as(kj_mol)

    df = pd.concat(
        [
            df,
            pd.DataFrame.from_dict(
                {
                    "Conformer No.": [idx],
                    "Toolkit (kJ/mol)": [round(toolkit, 3)],
                    "Interchange -> OpenMM (kJ/mol)": [round(omm, 3)],
                    "Interchange -> GROMACS": [round(gmx, 3)],
                }
            ),
            # ignore_index=True,
        ]
    )

In [ ]:
df.style.hide(axis="index")